# Decision Tree

## Prerequisite

In [1]:
import re
import pandas as pd;
import math
from sklearn.tree import DecisionTreeClassifier as dt
from sklearn.tree import plot_tree
import matplotlib

## Basic Classes

In [2]:
class Node:
    """ Node class for a decision tree. """
    def __init__(self, names):
        self.names = names

    def classify(x):
        """ Handled by the subclasses. """
        return None

    def dump(self, indent):
        """ Handled by the subclasses. """
        return None


class Leaf(Node):
    def __init__(self, names, value):
        Node.__init__(self, names)
        self.value = value

    def classify(self, x):
        return self.value

    def dump(self, indent):
        print(' %d' % self.value)


class Split(Node):
    def __init__(self, names, var, left, right):
        Node.__init__(self, names)
        self.var = var
        self.left = left
        self.right = right

    def classify(self, x):
        if x[self.var] == 0:
            return self.left.classify(x)
        else:
            return self.right.classify(x)
      
    def dump(self, indent):
        if indent > 0:
            print('')
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 0 :' % self.names[self.var],end='')
        self.left.dump(indent+1)
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 1 :' % self.names[self.var],end='')
        self.right.dump(indent+1)


Helper function computes entropy of Bernoulli distribution with parameter p

In [3]:
def entropy(p):

    if p == 0 or p == 1:
        return 0
    
    result = -(p * math.log2(p) + (1 - p) * math.log2(1 - p))
    
    return result;


Compute information gain for a particular split, given the counts 

py_pxi : number of occurences of y=1 with x_i=1 for all i=1 to n

pxi : number of occurrences of x_i=1

py : number of ocurrences of y=1


In [4]:
def infogain(py_pxi, pxi, py, total):

    Hs = entropy(py/total)
    if pxi == 0:
        H1 = 0
    else:
        H1 = entropy(py_pxi/pxi)
    if pxi == total:
        H0 = 0
    else:
        H0 = entropy((py - py_pxi)/(total - pxi))
    result = Hs - (pxi/total) * H1 - ((total - pxi)/total) * H0
        
    return result;


OTHER SUGGESTED HELPER FUNCTIONS:

-collect counts for each variable value with each class label

-find the best variable to split on, according to mutual information

-partition data based on a given variable	
	


In [5]:
# Load data from a file
def read_data(filename):
    f = open(filename, 'r')
    p = re.compile(',')
    data = []
    header = f.readline().strip()
    varnames = p.split(header)
    namehash = {}
    for l in f:
        data.append([int(x) for x in p.split(l.strip())])
    return (data, varnames)


Build tree in a top-down manner, selecting splits until we hit a pure leaf or all splits look bad.


In [6]:
def build_tree(data, varnames):
    
    dataSize = len(data)
    
    # if there's only one class left, the current node will be a leaf node
    py = 0
    for v in data:
        if v[-1] == 1:
            py += 1
    if py == 0:
        return Leaf(varnames, 0)
    elif py == dataSize:
        return Leaf(varnames, 1)

    
    # otherwise, the different info gains will be checked
    maxGain = -1
    maxFeatureIndex = -1

    for i in range(len(varnames) - 1):
        if varnames[i] != None:
            py_pxi = 0
            pxi = 0
            py = 0
            for v in data:
                if v[i] == 0:
                    if v[-1] == 1:
                        py += 1
                else:
                    if v[-1] == 1:
                        py_pxi += 1
                        py += 1
                    pxi += 1
            # calculate the info gain based on the current feature, and update the maximum info gain and corresponding feature index
            currentGain = infogain(py_pxi, pxi, py, dataSize)
            if currentGain > maxGain:
                maxGain = currentGain
                maxFeatureIndex = i
        
    # divide the data set based on the selected feature
    leftData = []
    rightData = []
    
    for v in data:
        if v[maxFeatureIndex] == 0:
            leftData.append(v)
        else:
            rightData.append(v)
    
    # disable the features that have been checked
    leftNames = varnames.copy()
    rightNames = varnames.copy()
    leftNames[maxFeatureIndex] = None
    rightNames[maxFeatureIndex] = None
    
    # build the subtrees
    leftNode = build_tree(leftData, leftNames)
    rightNode = build_tree(rightData, rightNames)
    # return the current node after building the subtrees
    return Split(varnames, maxFeatureIndex, leftNode, rightNode)



## Main Program

Here we load data.
Each example is a list of attribute values, where the last element in the list is the class value.


In [7]:
(train, varnames) = read_data('agaricuslepiotatrain1.csv')
(test, testvarnames) = read_data('agaricuslepiotatest1.csv')
#(train, varnames) = read_data('training_set.csv')
#(test, testvarnames) = read_data('test_set.csv') 

In [8]:
pd.DataFrame(train).head()

,0,1,2,3,4,5,6,7,8,9,...,113,114,115,116,117,118,119,120,121,122
0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
1,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
4,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [9]:
#varnames

Build the decision tree

In [10]:
root = build_tree(train, varnames)

In [11]:
root.dump(0)



odor-foul = 0 :
| gill-size-broad = 0 :
| | odor-none = 0 :
| | | gill-spacing-close = 0 :
| | | | bruises?-bruises = 0 : 1
| | | | bruises?-bruises = 1 : 0
| | | gill-spacing-close = 1 : 1
| | odor-none = 1 :
| | | stalk-surface-above-ring-silky = 0 :
| | | | bruises?-bruises = 0 : 0
| | | | bruises?-bruises = 1 : 1
| | | stalk-surface-above-ring-silky = 1 : 1
| gill-size-broad = 1 :
| | spore-print-color-green = 0 : 0
| | spore-print-color-green = 1 : 1
odor-foul = 1 : 1


Calcuating the accuracy

In [12]:
def accuracy(data):
    correct = 0
    # The position of the class label is the last element in the list.
    yi = len(data[0]) - 1
    for x in data:
    # Classification is done recursively by the node class.
    # This should work as-is.
        pred = root.classify(x)
        if pred == x[yi]:
            correct += 1
        acc = float(correct)/len(data)
    return acc;
   

## Results

In [13]:
 print("Train Accuracy: {}".format(accuracy(train)))

Train Accuracy: 1.0


In [14]:
 print("Test Accuracy: {}".format(accuracy(test)))

Test Accuracy: 0.9792941176470589
